In [3]:
! pip install -r environment/requirenments.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
import sys

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('data/')
sys.path.append(str('AI4Code'))

In [5]:
from ipynb.fs.full.utils import read_train_data, get_df_ranks, get_ancestors

In [ ]:
df = read_train_data(data_dir)
df_ranks = get_df_ranks(df, data_dir)

Train NBs: 100%|██████████| 10000/10000 [02:11<00:00, 75.85it/s]
